In [2]:
from google.colab import files
uploaded=files.upload()

Saving top_100_saas_companies_2025.csv to top_100_saas_companies_2025.csv


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, trim, col

# Start Spark session
spark = SparkSession.builder.appName("Investor Tier Enrichment").getOrCreate()

# Load your main CSV file into a Spark DataFrame
df = spark.read.csv("/content/top_100_saas_companies_2025.csv", header=True, inferSchema=True)

# Extract first investor from "Top Investors"
df = df.withColumn("First_Investor", trim(split(col("Top Investors"), ",").getItem(0)))

# Create investor_tiers table
investor_tiers = spark.createDataFrame([
    ("Accel", "Tier 1"),
    ("Sequoia", "Tier 1"),
    ("Andreessen Horowitz", "Tier 1"),
    ("SoftBank", "Tier 2"),
    ("Lightspeed", "Tier 2"),
    ("Unknown", "Tier 3")
], ["Investor", "Tier"])

#Join on investor name
joined_df = df.join(investor_tiers, df.First_Investor == investor_tiers.Investor, how='left')

# Filter for Tier 1 and Tier 2 investors
filtered_df = joined_df.filter(col("Tier").isin("Tier 1", "Tier 2"))

# Select required columns
result_df = filtered_df.select("Company Name", "First_Investor", "Tier", "Valuation")

#  result
result_df.show(truncate=False)


+------------------+-------------------+------+--------------------+
|Company Name      |First_Investor     |Tier  |Valuation           |
+------------------+-------------------+------+--------------------+
|Algolia           |Accel              |Tier 1|$2.3B               |
|Sumo Logic        |Accel              |Tier 1|$2.3B               |
|Segment           |Accel              |Tier 1|$3.2B (Twilio)      |
|Freshworks        |Accel              |Tier 1|$5.2B               |
|Slack             |Accel              |Tier 1|$27.7B (Salesforce) |
|Netskope          |Sequoia            |Tier 1|$7.5B               |
|Verkada           |Sequoia            |Tier 1|$3.2B               |
|Gong              |Sequoia            |Tier 1|$7.3B               |
|Qualtrics         |Sequoia            |Tier 1|$12.5B (Silver Lake)|
|RingCentral       |Sequoia            |Tier 1|$5B                 |
|Palo Alto Networks|Sequoia            |Tier 1|$95B                |
|Amplitude         |Sequoia       